## Catalysis: Water absorption

Catalysis is the increase of rate of a chemical reaction in the presence of an added subastance known as catalyst.

Catalyst are not part of the reagents nor the products, which has the advantage that only small quantites are required to be effective. Here, we will study the decomposition of a $H_2O$ molecule in the presence of a catalytic surface known as $Pt(111)$.

### Building your simulation structure: $H_{2}O$ molecule on a $Pt(111)$ catalytic surface 

The first step is building a slab of $Pt$ atoms and we will add a $H_2O$ molecule on top of the slab.

In [ ]:
#Create the atomic structure

from ase.build import fcc111, molecule, add_adsorbate
from ase.constraints import FixAtoms
from ase.visualize import view

atoms = fcc111('Pt', (3, 3, 4), vacuum=7.5)

add_adsorbate(atoms, molecule('H2O'), height=4.0,position=(6,4))

atoms.set_constraint([FixAtoms(mask=[atom.tag > 2 for atom in atoms])])

atoms.set_pbc([1,1,1])

view(atoms)

### Molecular dynamics simulations

We can use molecular dynamics to simulate how the $H_2O$ molecule moves in a real-time simulation. Notice how easily the water molecule atoms separate in the presence of $Pt(111)$.

In [ ]:
#Perform a molecular dynamics simulation
from ase.io.trajectory import Trajectory
from ase.calculators.emt import EMT
from ase.md.velocitydistribution import MaxwellBoltzmannDistribution
from ase.md import Langevin
from ase import units
from ase import Atoms
import ase.io

atoms = fcc111('Pt', (3, 3, 4), vacuum=7.5)

add_adsorbate(atoms, molecule('H2O'), height=4.0,position=(6,4))

atoms.set_constraint([FixAtoms(mask=[atom.tag > 2 for atom in atoms])])

atoms.set_pbc([1,1,1])

Temperature = 500#Temperature of study in Kelvin

atoms.calc = EMT()

MaxwellBoltzmannDistribution(atoms, temperature_K=Temperature)

dyn = Langevin(atoms, 1 * units.fs, friction=0.01, temperature_K=Temperature)

traj = Trajectory('traj/pt.traj', 'w', atoms)
dyn.attach(traj.write, interval=1)

# Now run the dynamics
dyn.run(500)

frames = ase.io.read('traj/pt.traj', index=':')
view(frames)

### Quantifying the dissociation energy

Likewise, we want to study the energetics involved in the reaction. In the box below, we start by optimizing the position of the atoms to obtained the most stable atomic configuration.

In [ ]:
from ase.build import add_adsorbate, fcc100
from ase.calculators.emt import EMT
from ase.constraints import FixAtoms
from ase.optimize import LBFGS
from ase.io import read, write

# 2x2-Al(001) surface with 3 layers and an
# Au atom adsorbed in a hollow site:
atoms = fcc111('Pt', (3, 3, 4), vacuum=7.5)

add_adsorbate(atoms, molecule('H2O'), height=2.0,position=(6,4))

atoms.set_constraint([FixAtoms(mask=[atom.tag > 2 for atom in atoms])])

atoms.set_pbc([1,1,1])


# Use EMT potential:
atoms.calc = EMT()

# Initial state
opt = LBFGS(atoms)
opt.run(fmax=0.01)

# Final state:
atoms[-1].position[0] = atoms[-2].position[0] - 0.25 * atoms.cell[0, 0]
atoms[-1].position[1] = atoms[-2].position[0] - 0.5 * atoms.cell[0, 0]


# Use EMT potential:
atoms.calc = EMT()

# Initial state
opt = LBFGS(atoms)
opt.run(fmax=0.01)
write('traj/initial.traj', atoms)

view(atoms)

Known what is the most stable configuration for the water molecule, we want to know what is the energy of a final state in which one of the hydrogen atoms has been separated from the oxygen atom. To do this, we manually change the position of one of the hydrogen atoms and relax the structure so that it ends in a energy minimum.

In [ ]:

# Final state:
atoms[-1].position[0] = atoms[-2].position[0] - 0.5 * atoms.cell[0, 0]
atoms[-1].position[1] = atoms[-2].position[0] - 0.625 * atoms.cell[0, 0]


opt.run(fmax=0.01)
print('final state:', atoms.get_potential_energy())
write('traj/final.traj', atoms)

# and relax.
view(atoms)

Lastly, we want to study the energy barrier that the hydrogen atom needs to overcome in order to reach the final state from its initial state. 

This is done through a technique know as nudge elastic band (NEB) method.

In [ ]:
from ase.calculators.emt import EMT
from ase.constraints import FixAtoms
from ase.io import read
from ase.neb import NEB
from ase.optimize import LBFGS

initial = read('traj/initial.traj')
final = read('traj/final.traj')

constraint = FixAtoms(mask=[atom.tag > 1 for atom in initial])

images = [initial]
for i in range(9):
    image = initial.copy()
    image.calc = EMT()
    image.set_constraint(constraint)
    images.append(image)


images.append(final)

neb = NEB(images)
neb.interpolate(apply_constraint=True)
qn = LBFGS(neb, trajectory='traj/neb.traj')
qn.run(fmax=0.01)

frames = ase.io.read('traj/neb.traj', index=':')
view(frames)

Once the method has completed, we can plot the results.

In [ ]:
import matplotlib.pyplot as plt

from ase.io import read
from ase.neb import NEBTools

images = read('traj/neb.traj@-9:')

nebtools = NEBTools(images)

# Get the calculated barrier and the energy change of the reaction.
Ef, dE = nebtools.get_barrier()

# Get the barrier without any interpolation between highest images.
Ef, dE = nebtools.get_barrier(fit=False)

# Get the actual maximum force at this point in the simulation.
max_force = nebtools.get_fmax()

# Create a figure like that coming from ASE-GUI.
fig = nebtools.plot_band()
fig.show()
